In [22]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam, SGD, Adagrad, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, Callback
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore', category=DeprecationWarning)

#### Preprocessing

In [23]:
train, test = [], []
z_train, z_test = [], []
# chars =  ['a', 'bA', 'dA', 'lA', 'tA']
chars = ["a", "ai", "bA", "lA", "tA"]

def normalize(seq):
    min_x, max_x = min(el[0] for el in seq), max(el[0] for el in seq)
    min_y, max_y = min(el[1] for el in seq), max(el[1] for el in seq)
    for el in seq:
        el[0] = (el[0] - min_x)/(max_x-min_x)
        el[1] = (el[1] - min_y)/(max_y-min_y)
    return seq
def read_ip(path):
    seq = []
    with open(path, 'r') as file:
        contents = file.read()
        items = contents.split()
        for i in range(1,len(items),2):
            seq.append([float(items[i]),float(items[i+1])])
        train.append(normalize(seq))
    return seq

for ch in chars:
    for f in os.listdir("Group24/Handwriting_Data/" + str(ch)+"/train"):
        with open("Group24/Handwriting_Data/" + str(ch)+"/train/" + str(f), 'r') as file:
            contents = file.read()
            items = contents.split()
            seq = []
            for i in range(1,len(items),2):
                seq.append([float(items[i]),float(items[i+1])])
            train.append(normalize(seq))
        z_train.append(str(ch))
    for f in os.listdir("Group24/Handwriting_Data/" + str(ch)+"/dev"):
        with open("Group24/Handwriting_Data/" + str(ch)+"/dev/" + str(f), 'r') as file:
            contents = file.read()
            items = contents.split()
            seq = []
            for i in range(1,len(items),2):
                seq.append([float(items[i]),float(items[i+1])])
            test.append(normalize(seq))
        z_test.append(str(ch))

In [24]:
# label_map = {'a':0, 'bA':1, 'dA':2, 'lA':3, 'tA':4}
label_map = {"a":0, "ai":1, "bA":2, "lA":3, "tA":4}
z_train_onehot = tf.keras.utils.to_categorical([label_map[x] for x in z_train], 5)
z_test_onehot = tf.keras.utils.to_categorical([label_map[x] for x in z_test], 5)

Flattening

#### Model

In [25]:
my_tensor = tf.ragged.constant(train)
model = Sequential()
model.add(SimpleRNN(32, batch_input_shape=(None, None, 2), return_sequences=True))
# model.add(SimpleRNN(32, activation="relu", return_sequences=True))
# model.add(SimpleRNN(64, activation="relu", return_sequences=True))
model.add(SimpleRNN(32, activation="relu", return_sequences=False))
model.add(Dense(32, activation="relu"))
model.add(Dense(5, activation="softmax"))
loss_fn = tf.keras.losses.CategoricalCrossentropy()

adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss="categorical_crossentropy", optimizer=adam_optimizer, metrics=['accuracy']) 
# Train the model
my_callbacks = [ EarlyStopping(monitor='loss', min_delta=1e-4, patience=10),    TensorBoard(log_dir=f'./logdir/Q1/test')]
model_fit = model.fit(my_tensor, z_train_onehot, batch_size=len(train), epochs=10000, verbose=1, validation_split=0.0,callbacks=my_callbacks, shuffle=False, validation_batch_size=None)

hist_metric = 'accuracy'
print(f'epochs: {len(model_fit.history[hist_metric])}, acc: {model_fit.history[hist_metric][-1]}\n')
model.save(f'models/Q1/test.tf')

test_tensor = tf.ragged.constant(test)
loss, accuracy = model.evaluate(test_tensor,z_test_onehot)
print("Test loss = ", loss)
print("Test accuracy = ", accuracy)

Epoch 1/10000
1/1 [==============================] - 1s 1s/step - loss: 1.7270 - accuracy: 0.1720
Epoch 2/10000
1/1 [==============================] - 0s 64ms/step - loss: 1.6797 - accuracy: 0.1866
Epoch 3/10000
1/1 [==============================] - 0s 69ms/step - loss: 1.6393 - accuracy: 0.1953
Epoch 4/10000
1/1 [==============================] - 0s 64ms/step - loss: 1.6030 - accuracy: 0.2216
Epoch 5/10000
1/1 [==============================] - 0s 66ms/step - loss: 1.5920 - accuracy: 0.3848
Epoch 6/10000
1/1 [==============================] - 0s 65ms/step - loss: 1.5793 - accuracy: 0.3032
Epoch 7/10000
1/1 [==============================] - 0s 63ms/step - loss: 1.5647 - accuracy: 0.2711
Epoch 8/10000
1/1 [==============================] - 0s 65ms/step - loss: 1.5534 - accuracy: 0.2945
Epoch 9/10000
1/1 [==============================] - 0s 61ms/step - loss: 1.5450 - accuracy: 0.3499
Epoch 10/10000
1/1 [==============================] - 0s 62ms/step - loss: 1.5336 - accuracy: 0.3936
E

Fastest : 32(rnn),32(rnn),32(dense),5(dense)